# Python function to assign air pollution levels to each local authority level

## Part of the article: Links between air pollution and COVID-19 in England.

Link to the preprint of the paper: https://www.medrxiv.org/content/10.1101/2020.04.16.20067405v3.full

**Author:** Yizhou Yu<br>**Affiliation:** MRC Toxicology Unit, University of Cambridge

Date: 19 August 2020

In this python script, I first use a free version of OpenCageGeocode (https://opencagedata.com/api) to generate the official longitude and latitude of each. local authority in England. Then, I use a function in sklearn to find the 10 nearest modelled air pollutant value based on center of each city. Finally, I match the averaged value to each local authority. 

In [13]:
import pandas as pd                  # for DataFrames
from opencage.geocoder import OpenCageGeocode
key = '7890eb0385704a3ea508afa9f38363e1'
geocoder = OpenCageGeocode(key)

Import the match function from the match_air_to_UKB_covid. Set the closest value of air pollutant as the average of 10 nearest points.

In [14]:
#build functions
from sklearn.neighbors import BallTree
import numpy as np

def get_nearest(src_points, candidates, k_neighbors=10):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()
    radius = distances[9]

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest0 = indices[0]    
    closest1 = indices[1]
    closest2 = indices[2]
    closest3 = indices[3]
    closest4 = indices[4]
    closest5 = indices[5]
    closest6 = indices[6]
    closest7 = indices[7]
    closest8 = indices[8]
    closest9 = indices[9]
    #return (closest9)
    # Return indices and distances
    return (radius, closest0, closest1,closest2,closest3,closest4,closest5,closest6,closest7,closest8,closest9)


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    radius, closest, closest1,closest2,closest3,closest4,closest5,closest6,closest7,closest8,closest9 = get_nearest(src_points=left_radians, candidates=right_radians)
    #closest = get_nearest(src_points=left_radians, candidates=right_radians)
    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest].reset_index(drop=True)
    closest_points1 = right.loc[closest1].reset_index(drop=True)
    closest_points2 = right.loc[closest2].reset_index(drop=True)
    closest_points3 = right.loc[closest3].reset_index(drop=True)
    closest_points4 = right.loc[closest4].reset_index(drop=True)
    closest_points5 = right.loc[closest5].reset_index(drop=True)
    closest_points6 = right.loc[closest6].reset_index(drop=True)
    closest_points7 = right.loc[closest7].reset_index(drop=True)
    closest_points8 = right.loc[closest8].reset_index(drop=True)
    closest_points9 = right.loc[closest9].reset_index(drop=True)
    
    df_to_agg = pd.concat([closest_points, closest_points1,
                          closest_points2,
                          closest_points3,
                          closest_points4,
                          closest_points5,
                          closest_points6,
                          closest_points7,
                          closest_points8,
                          closest_points9])
    df_to_agg = df_to_agg.drop(columns = ['geometry'])
    df_agg = df_to_agg.groupby(df_to_agg.index).agg('mean')
    
    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        df_agg['radius'] = radius * earth_radius

    merged_gpd = left_gdf.join(df_agg)
    return merged_gpd

In [15]:
covid_dt = pd.read_csv("data_output_v4/merged_covid_cov_dt_LA.csv")
covid_dt

,Unnamed: 0,Code,deaths,X2018.people.per.sq..km,Mean_ann_earnings,median_age_2018,Name
0,1,E06000001,20,997,25985.0,41.8,Hartlepool
1,2,E06000002,60,2608,22878.0,36.2,Middlesbrough
2,3,E06000003,26,558,23236.0,45.0,Redcar and Cleveland
3,4,E06000004,26,962,26622.0,40.4,Stockton-on-Tees
4,5,E06000005,14,540,26908.0,43.1,Darlington
...,...,...,...,...,...,...,...
329,330,W06000020,33,740,25755.0,42.4,Torfaen
330,331,W06000021,25,111,32558.0,48.6,Monmouthshire
331,332,W06000022,61,805,24974.0,38.8,Newport
332,333,W06000023,18,26,22421.0,49.9,Powys


In [16]:
#loop to get lat long
list_lat = []   # create empty lists

list_lon = []

for index, row in covid_dt.iterrows(): # iterate over rows in dataframe
    
    query = row['Name']  + ', England, UK'

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    lon = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_lon.append(lon)

# create new columns from lists    

covid_dt['lat'] = list_lat   

covid_dt['lon'] = list_lon

In [17]:
covid_dt.head()

,Unnamed: 0,Code,deaths,X2018.people.per.sq..km,Mean_ann_earnings,median_age_2018,Name,lat,lon
0,1,E06000001,20,997,25985.0,41.8,Hartlepool,54.685728,-1.209370
1,2,E06000002,60,2608,22878.0,36.2,Middlesbrough,54.576042,-1.234405
2,3,E06000003,26,558,23236.0,45.0,Redcar and Cleveland,54.567906,-1.005496
3,4,E06000004,26,962,26622.0,40.4,Stockton-on-Tees,54.564094,-1.312916
4,5,E06000005,14,540,26908.0,43.1,Darlington,54.524208,-1.555581


In [18]:
covid_dt.lat.describe()

count    334.000000
mean      52.310860
std        1.115308
min       49.920341
25%       51.454608
50%       52.148415
75%       53.166456
max       55.250000
Name: lat, dtype: float64

In [19]:
covid_dt.lon.describe()

count    334.000000
mean      -1.223613
std        1.319393
min       -6.292879
25%       -2.155556
50%       -1.143365
75%       -0.212901
max        1.731077
Name: lon, dtype: float64

In [20]:
covid_dt.to_csv("data_output_v4/merged_covid_cov_dt_LA_LL.csv", index=False)

Upon curation, these location are very good as they target the center of the 


## Now, merge the covid dataset to the modelled background pollution dataset 

In [27]:
#these are the same datasets that I curated and used for the uk biobank analysis
pm25_df = pd.read_csv("data_v4/processed_pm25_lonlat.csv", usecols=['pm25_lon','pm25_lat','pm25_val'])
no2_df = pd.read_csv('data_v4/processed_no2_lonlat.csv', usecols = ['no2_lon','no2_lat','no2_val'])
o3_df = pd.read_csv('data_v4/processed_o3_lonlat.csv', usecols = ['o3_lon','o3_lat','o3_val'])
pm10_df = pd.read_csv('data_v4/processed_pm10_lonlat.csv', usecols = ['pm10_lon','pm10_lat','pm10_val'])
so2_df = pd.read_csv('data_v4/processed_so2_lonlat.csv', usecols = ['so2_lon','so2_lat','so2_val'])
nox_df = pd.read_csv('data_v4/processed_nox_lonlat.csv', usecols = ['nox_lon','nox_lat','nox_val'])

In [60]:
covid_dt = pd.read_csv('data_v4/merged_covid_cov_dt_LA_LL.csv')

In [30]:
import geopandas as gpd
def to_gpd_air_dt(input_df):
    '''
    from pd to gpd 
    the input datasets must be lon, lat, val
    '''
    out_gpd = gpd.GeoDataFrame(
    input_df, geometry=gpd.points_from_xy(input_df.iloc[:,0], input_df.iloc[:,1]))
    return(out_gpd)

In [31]:
pm25_gpd = to_gpd_air_dt(pm25_df)
no2_gpd = to_gpd_air_dt(no2_df)
o3_gpd = to_gpd_air_dt(o3_df)
pm10_gpd = to_gpd_air_dt(pm10_df)
so2_gpd = to_gpd_air_dt(so2_df)
nox_gpd = to_gpd_air_dt(nox_df)


In [194]:
pm25_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm25_gpd, return_dist=True)
no2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], no2_gpd, return_dist=True)
o3_covid = nearest_neighbor(covid_gpd[['Code','geometry']], o3_gpd, return_dist=True)
pm10_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm10_gpd, return_dist=True)
so2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], so2_gpd, return_dist=True)
nox_covid = nearest_neighbor(covid_gpd[['Code','geometry']], nox_gpd, return_dist=True)

In [195]:
# now i need to match the df above
pm25_covid.head()

,pm25_lon,pm25_lat,pm25_val,radius
0,-1.210453,54.684977,7.362821,2313.328476
1,-1.234211,54.576387,8.210540,2140.177884
2,-1.005466,54.567534,7.293137,2144.162968
3,-1.310219,54.564262,7.900856,2274.218771
4,-1.556564,54.523176,7.153221,2342.234609
...,...,...,...,...
329,-0.704773,52.159159,9.543746,2236.378453
330,-2.768916,51.480431,8.189910,2098.695090
331,-1.317599,50.689653,8.870632,2196.216171
332,-3.242789,50.685021,5.966326,2155.897392


In [197]:
# This code is now obsolete
# pm25_covid = covid_gpd[['Code','geometry']].join(pm25_covid)
# no2_covid = covid_gpd[['Code','geometry']].join(no2_covid)
# o3_covid = covid_gpd[['Code','geometry']].join(o3_covid)
# pm10_covid = covid_gpd[['Code','geometry']].join(pm10_covid)
# so2_covid = covid_gpd[['Code','geometry']].join(so2_covid)
# nox_covid = covid_gpd[['Code','geometry']].join(nox_covid)
# so2_covid

,Code,geometry,so2_lon,so2_lat,so2_val,radius
0,E06000001,POINT (-1.20937 54.68573),-1.210453,54.684977,1.575561,2313.328476
1,E06000002,POINT (-1.23440 54.57604),-1.234211,54.576387,3.133286,2140.177884
2,E06000003,POINT (-1.00550 54.56791),-1.005466,54.567534,0.998864,2144.162968
3,E06000004,POINT (-1.31292 54.56409),-1.310219,54.564262,1.934330,2274.218771
4,E06000005,POINT (-1.55558 54.52421),-1.556564,54.523176,1.129034,2342.234609
...,...,...,...,...,...,...
329,W06000020,POINT (-0.70312 52.16045),-0.704773,52.159159,1.014256,2236.378453
330,W06000021,POINT (-2.76885 51.48187),-2.768916,51.480431,1.465795,2098.695090
331,W06000022,POINT (-1.31636 50.69131),-1.317599,50.689653,1.229098,2196.216171
332,W06000023,POINT (-3.24297 50.68275),-3.242789,50.685021,0.607522,2155.897392


In [54]:
no2_covid.head()

,Code,geometry,no2_5yAvg,lon,lat,radius
0,E06000001,POINT (-1.20937 54.68573),14.359226,-1.210453,54.684977,2.313328e+03
1,E06000002,POINT (-1.23440 54.57604),20.537653,-1.234211,54.576387,2.140178e+03
2,E06000003,POINT (-1.00550 54.56791),8.440613,-1.005466,54.567534,2.144163e+03
3,E06000004,POINT (-1.31292 54.56409),16.408541,-1.310219,54.564262,2.274219e+03
4,E06000005,POINT (-1.55558 54.52421),12.552449,-1.556564,54.523176,2.342235e+03
...,...,...,...,...,...,...
329,W06000020,POINT (-3.05205 51.70148),9.630536,-3.053421,51.703012,2.217284e+03
330,W06000021,POINT (-2.83333 51.75011),7.004439,-2.831425,51.749750,2.160823e+03
331,W06000022,POINT (-71.31377 41.48998),2.154993,-8.626371,57.826835,7.064841e+06
332,W06000023,POINT (-3.35509 52.32715),3.461820,-3.355866,52.329299,2.179994e+03


In [58]:
#merge everything togerther and export 

covid_Allair_poll_df = pd.merge(covid_gpd.drop(columns=['geometry','Name']), pm25_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, no2_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, o3_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, pm10_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, so2_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, nox_covid.drop(columns=['geometry','radius']), on='Code')

covid_Allair_poll_df.to_csv("data_output_v4/merged_covidAir_cov_dt_LA.csv", index=False)

# Repeat this process for 5 year average air pollution values 

In [39]:
# I will overwrite on the same variables for simplicity
covid_dt = pd.read_csv('data_output_v4/merged_covid_cov_dt_LA_LL.csv')
# AP5Y_dt = pd.read_csv('data_output_v4/5Yaverage_AP_PCMdata_na.csv')
pm25_df = pd.read_csv("data_output_v4/processed_allAP_lonlat.csv", usecols=['lon','lat','pm25_5yAvg'])
no2_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','no2_5yAvg'])
o3_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','o3_5yAvg'])
pm10_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','pm10_5yAvg'])
# so2_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','so2_val'])
nox_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','nox_5yAvg'])


In [40]:
import geopandas as gpd
def to_gpd_air_dt(input_df):
    '''
    from pd to gpd 
    the input datasets must be lon, lat, val
    '''
    out_gpd = gpd.GeoDataFrame(
    input_df, geometry=gpd.points_from_xy(input_df.loc[:,'lon'], input_df.loc[:,'lat']))
    return(out_gpd)

In [41]:
# annotate data type
pm25_gpd = to_gpd_air_dt(pm25_df)
no2_gpd = to_gpd_air_dt(no2_df)
o3_gpd = to_gpd_air_dt(o3_df)
pm10_gpd = to_gpd_air_dt(pm10_df)
# so2_gpd = to_gpd_air_dt(so2_df)
nox_gpd = to_gpd_air_dt(nox_df)

In [42]:
covid_dt = pd.read_csv('data_v4/merged_covid_cov_dt_LA_LL.csv')
covid_gpd = to_gpd_air_dt(covid_dt)

In [50]:
#Calculations here 
pm25_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm25_gpd, return_dist=True)
no2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], no2_gpd, return_dist=True)
o3_covid = nearest_neighbor(covid_gpd[['Code','geometry']], o3_gpd, return_dist=True)
pm10_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm10_gpd, return_dist=True)
# so2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], so2_gpd, return_dist=True)
nox_covid = nearest_neighbor(covid_gpd[['Code','geometry']], nox_gpd, return_dist=True)

In [60]:
#merge everything togerther and export 

covid_Allair_poll_df = pd.merge(covid_gpd.drop(columns=['geometry','Name','lon','lat']), 
                                pm25_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                no2_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                o3_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                pm10_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
# covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, so2_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                nox_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')

covid_Allair_poll_df = covid_Allair_poll_df.drop(columns=['Unnamed: 0'])
covid_Allair_poll_df.to_csv("data_output_v4/merged_covidAir_cov_LA_5YA.csv", index=False)

In [61]:
covid_Allair_poll_df.head()

,Code,deaths,X2018.people.per.sq..km,Mean_ann_earnings,median_age_2018,pm25_5yAvg,no2_5yAvg,o3_5yAvg,pm10_5yAvg,nox_5yAvg
0,E06000001,20,997,"25,985",41.8,7.617912,14.359226,2.811139,11.625292,20.206840
1,E06000002,60,"2,608","22,878",36.2,8.342286,20.537653,2.240947,12.522083,30.628742
2,E06000003,26,558,"23,236",45.0,7.748073,8.440613,3.795747,12.845976,11.273613
3,E06000004,26,962,"26,622",40.4,8.038180,16.408541,2.670293,12.169144,23.435384
4,E06000005,14,540,"26,908",43.1,7.368714,12.552449,2.763807,11.199637,17.361185


#### References

API design for machine learning software: experiences from the scikit-learn project, Buitinck et al., 2013.

Data structures for statistical computing in python, McKinney, Proceedings of the 9th Python in Science Conference, Volume 445, 2010.



